202001555 지은미

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

In [4]:
loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('../data', 
                   train=True, 
                   download=True, 
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor()])),
    batch_size=32,
    num_workers=0,
    shuffle=False)

mean = 0.
mean_square = 0.
samples = 0
for images, _ in loader:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    mean_square += (images**2).mean(2).sum(0)
    samples += images.size(2) * images.size(0)

mean /= len(loader.dataset)
mean_square /= len(loader.dataset)

# extra scale factor for unbias std estimate (it's effectively 1.0)
scale = samples / (samples - 1)
std = torch.sqrt((mean_square - mean**2) * scale)

print("The mean is ", mean)
print("The standard deviation is ", std)

The mean is  tensor([0.2860])
The standard deviation is  tensor([0.3530])


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((mean,), (std,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((mean,), (std,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [7]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319977
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.588379
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.911626
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.955980
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.017538
[1] Test Loss: 0.7271, Accuracy: 72.17%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.750698
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.907721
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.674810
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.791311
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.587482
[2] Test Loss: 0.6132, Accuracy: 76.81%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.838469
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.679400
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.943937
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.678740
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.548743
[3] Test Loss: 0.5587, Accuracy: 77.84%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.711359
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.694363
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.560752


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.559397
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.390347
[29] Test Loss: 0.3332, Accuracy: 88.04%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.507118
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.358356
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.458844
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.485223
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.313737
[30] Test Loss: 0.3352, Accuracy: 87.93%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.526629
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.216413
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.288969
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.380071
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.335202
[31] Test Loss: 0.3340, Accuracy: 88.09%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.344233
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.419559
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.379910
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.502644
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.663314
[32] Test Loss: 0.3290, A

202001555 지은미